In [1]:
# importing libraries
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE, RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import precision_recall_cutoff

# defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'webster-data445-bucket'
bucket = s3.Bucket(bucket_name)

# defining the csv file
file_key = 'turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the csv file
turnover = pd.read_csv(file_content_stream)
# dropping missing values
turnover = turnover.dropna()
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
# changing sales to dummy variable
turnover = pd.concat([turnover.drop(columns = ['sales']), pd.get_dummies(turnover['sales'])], axis = 1)

# changing salary to dummy variable
turnover = pd.concat([turnover, pd.get_dummies(turnover['salary'])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical,high,low,medium
0,0.38,0.53,2,157,3,0,1,0,low,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,medium,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,medium,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,low,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,low,0,0,0,0,0,0,0,1,0,0,0,1,0


In [3]:
# creating interactions from the decision tree
turnover['interaction_1'] = np.where((turnover['satisfaction_level'] <= 0.465) & (turnover['number_project'] <= 2.5) & (turnover['last_evaluation'] <= 0.575), 1, 0)
turnover['interaction_2'] = np.where((turnover['satisfaction_level'] <= 0.465) & (turnover['number_project'] >= 2.5) & (turnover['satisfaction_level'] <= 0.115), 1, 0)
turnover['interaction_3'] = np.where((turnover['satisfaction_level'] >= 0.465) & (turnover['time_spend_company'] <= 4.5) & (turnover['average_montly_hours'] <= 290.5), 1, 0)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,salary,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical,high,low,medium,interaction_1,interaction_2,interaction_3
0,0.38,0.53,2,157,3,0,1,0,low,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0
1,0.80,0.86,5,262,6,0,1,0,medium,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,0.11,0.88,7,272,4,0,1,0,medium,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
3,0.72,0.87,5,223,5,0,1,0,low,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0.37,0.52,2,159,3,0,1,0,low,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0


In [6]:
# defining input and target variables
X = turnover.drop(columns = ['left', 'salary'], axis = 1)
Y = turnover['left']

# splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

# changing scale to 0-1
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [7]:
# running RFE with logistic
Logistic_auto_features = RFECV(estimator = LogisticRegression(), step = 1, min_features_to_select = 2, cv = 3).fit(X_train, Y_train)

# identifying important features
print(X_train.columns[Logistic_auto_features.support_])

Index(['satisfaction_level', 'last_evaluation', 'average_montly_hours',
       'time_spend_company', 'interaction_1', 'interaction_2',
       'interaction_3'],
      dtype='object')


In [8]:
# defining the inputs
X_train_1 = X_train[['satisfaction_level', 'last_evaluation', 'average_montly_hours',
       'time_spend_company', 'interaction_1', 'interaction_2',
       'interaction_3']]

X_test_1 = X_test[['satisfaction_level', 'last_evaluation', 'average_montly_hours',
       'time_spend_company', 'interaction_1', 'interaction_2',
       'interaction_3']]


# building logistic model
logit_md = LogisticRegression().fit(X_train_1, Y_train)

# prediction on test for logistic model
logit_pred = logit_md.predict_proba(X_test_1)[:, 1]

# predicting labels 
logit_labels = precision_recall_cutoff.precision_recall_cutoff(Y_test, logit_pred)

# computing the classification report
print(classification_report(Y_test, logit_labels))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2286
           1       0.95      0.91      0.93       714

    accuracy                           0.97      3000
   macro avg       0.96      0.95      0.96      3000
weighted avg       0.97      0.97      0.97      3000



In [10]:
# running RFE with random forest
RF_auto_feature = RFECV(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), step = 1, min_features_to_select = 2, cv = 3).fit(X_train, Y_train)

# identifying important features
print(X_train.columns[RF_auto_feature.support_])

Index(['satisfaction_level', 'interaction_1', 'interaction_3'], dtype='object')


In [12]:
# defining the inputs
X_train_2 = X_train[['satisfaction_level', 'interaction_1', 'interaction_3']]

X_test_2 = X_test[['satisfaction_level', 'interaction_1', 'interaction_3']]


# building random forest model
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train_2, Y_train)

# prediction on test for logistic model
RF_pred = RF_md.predict_proba(X_test_2)[:, 1]

# predicting labels 
RF_labels = precision_recall_cutoff.precision_recall_cutoff(Y_test, RF_pred)

# computing the classification report
print(classification_report(Y_test, RF_labels))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      2286
           1       0.88      0.92      0.90       714

    accuracy                           0.95      3000
   macro avg       0.93      0.94      0.94      3000
weighted avg       0.95      0.95      0.95      3000



In [ ]:
# The logistic regression model has better performance, but has many more variables. Because of this, I would choose the random forest
# approach as the difference in performance is small.